# Corsica Example

The [eqasim](https://github.com/eqasim-org/eqasim-java) repository is a packaged version of MATSim which makes use of discrete choice models instead of the scoring process for decision making. It uses a small simulation scenario for Corsica for unit testing. This scenario is packaged with the eqasim repository, so it can be used here easily for testing.

The following will show how to set up the use case (in this notebook) and then perform a basic calibration task.

<font color="red">Note that the calibration steps will take a while since the scenario, although it is small, still need to be run for 40 iterations and multiple times.</font>

## Use case

First, we clone the `eqasim` repository into `eqasim-java`. For that, `git` needs to be installed:

In [ ]:
# Only if you have git available
!git clone https://github.com/eqasim-org/eqasim-java

In case you don't have git installed on the command line, you can use other tools to clone the repository.

Next, we want to build the executable jar. For that, Maven needs to be installed on the command line. If you don't have it available, you can use an IDE like IntelliJ or Eclipse to do so:

In [ ]:
!cd eqasim-java && mvn -Pstandalone package --projects ile_de_france --also-make -DskipTests=true

The following code will find the latest executable jar generated by Maven:

In [ ]:
import glob, re

potential_jars = glob.glob("eqasim-java/ile_de_france/target/*.jar")
executable_jar = [
    jar for jar in potential_jars
    if re.search(r"/ile_de_france-[0-9.]+\.jar", jar)
]

if len(executable_jar) != 1:
    raise RuntimeError("Could not find executable jar")

executable_jar = executable_jar[0]
executable_jar

# Simulator

To run the calibration, we need to define the simulator. For MATSim, we already have a prepackaged version, which we create like this:

In [ ]:
import os, shutil
from octras.matsim import MATSimSimulator

if os.path.exists("eqasim_example_working_directory"):
    shutil.rmtree("eqasim_example_working_directory")

os.mkdir("eqasim_example_working_directory")

simulator = MATSimSimulator(
    working_directory = "eqasim_example_working_directory", # Working directory, which must exist.
    parameters = dict(
        class_path = executable_jar,
        main_class = "org.eqasim.ile_de_france.RunSimulation",
        iterations = 40,
        arguments = [
            "--config-path", 
            "eqasim-java/ile_de_france/src/main/resources/corsica/corsica_config.xml"
        ]
    )
)

As you see above, we create a working directory where the simulator will save all temporary data. Furthermore, we define some default arguments to the simulator (here based on eqasim).

We can test by running two iterations of this simulation. This takes a while:

In [ ]:
# Set up logging which we would normally see on the command line

import logging
logger = logging.getLogger("octras")
logger.setLevel(logging.DEBUG)

import time

# We define a run and override the "iterations" parameter:
print("Starting run ...")
simulator.run("my_test_run", parameters = dict(
    iterations = 2
))

# The simulation is now running in the background. We can wait for completion like so:
print("Still running ...")
while not simulator.ready("my_test_run"):
    time.sleep(1.0)
    
# Afterwards, as we have no need for them for the time being, we can clean the temporarily saved output data:
simulator.clean("my_test_run")
print("Finished!")

## Calibration Problem

We can now set up the calibration problem as class:

In [ ]:
from octras import Problem

import pandas as pd
import numpy as np

class CorsicaProblem(Problem):
    def __init__(self, reference_share):
        # Our goal is to calibrate the mode share of the "car" mode in the simulation while 
        # modifying the alternative-specific constant (ASC) of the mode. We make the reference
        # share configurable form outside the problem.
        self.reference_share = reference_share
        
    def get_information(self):
        # Here, we tell the optimizer that the problem has one parameter (the ASC)
        # and we provide the initial value for calibration, which can be used by
        # the optimizer (depending on which we choose). As we use only one parameters
        # (the ASC), we provide only one value.
        
        return dict(
            number_of_parameters = 1,
            initial_values = [0.0]
        )

    def parameterize(self, x):
        # This method translates a numeric list (x) into instructions for the simulator. In
        # this case, we use the generic MATSim simulator implementation. To append arguments to
        # the command line call of MATSim, we use the "arguments" return value. It will NOT override
        # the value from above, but instead also append these commands to the existing command.
        return dict(
            arguments = ["--mode-choice-parameter:car.alpha_u", x[0]]
        )

    def evaluate(self, x, path):
        # Here, we are should return an objective value. To guide the optimization, we calculate a 
        # deviation of the currently achieved mode share after the simulation with the reference
        # value. To do so, MATSim provides us the output path of the MATSim simulation where we can
        # access all the relevant results.
        
        # Read the modestats file as CSV
        mode_stats_paths = glob.glob("%s/modestats.txt" % path)
        df = pd.read_csv(mode_stats_paths[0], sep = "\t")

        # Get share of car trips in the last iteration
        simulation_share = df["car"].values[-1]

        # Construct the obejctive value
        objective = np.abs(simulation_share - self.reference_share)

        # Return objective
        return objective

## Optimization loop

Now, we can set up the optimization loop:

In [ ]:
from octras import Evaluator
from octras.algorithms import CMAES

# We already have the simulator
simulator

# We create an instance of the problem
problem = CorsicaProblem(reference_share = 0.3) # And we target 30% share

# We need to create an "evaluator" which is a generic bridge between the optimizers and the simulators
evaluator = Evaluator(
    problem = problem,
    simulator = simulator,
    parallel = 1 # Here we could say that we allow N simulations in parallel
)

# We need to set up the optimization algorithm, here CMA-ES
algorithm = CMAES(
    problem = problem,
    initial_step_size = 0.1,
    seed = 0
)

And we can run one optimization step with the CMA-ES algorithm. Note that this means that multiple evaluations will be performed on the simulator.

In [ ]:
# We can now perform one step of the algorithm. This may mean that it calls the evaluator multiple times with 
# different values, for instance for the estimation of gradients or similar tasks.
algorithm.advance(evaluator)

# Afterwards, we can obtain the trace of all the specific calls by calling fetch_trace on the evaluator.
# Those traces contain a lot of information provided by the problem, the simulator and the evaluator and 
# can be used for detailed and customized analysis.
trace = evaluator.fetch_trace()

# As a shortcut, we can also, for instance, just obtain the current number of evaluations
evaluator.current_evaluations

You can call the above block multiple times to advance the algorithm.

However, it is more convenient to automatically run in a loop. For that, we can set up a tracker which is notified whenever there is new results after evaluating the simulator:

In [ ]:
import numpy as np

# For plotting
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

class MyTracker:
    def __init__(self):
        # We track the best and current objective for plotting
        self.best_objectives = []
        self.current_objectives = []
        
        # Also, we track the proposed parameter values that correspond to them
        self.best_parameters = []
        self.current_parameters = []
        
    def notify(self, evaluation):
        # We are notified by each evaluation

        best_value = np.inf if len(self.best_objectives) == 0 else self.best_objectives[-1]
        best_parameter_value = np.nan if len(self.best_objectives) == 0 else self.best_parameters[-1]

        if evaluation["objective"] < best_value:
            best_value = evaluation["objective"]
            best_parameter_value = evaluation["x"][0]

        self.best_objectives.append(best_value)
        self.current_objectives.append(evaluation["objective"])
        
        self.best_parameters.append(best_parameter_value)
        self.current_parameters.append(evaluation["x"][0])
        
        self.plot()
        
    def plot(self):
        clear_output(wait = True)

        plt.figure(figsize = (10, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(self.current_objectives, 'ok')
        plt.plot(self.best_objectives, 'r')
        plt.xlabel("Evaluation")
        plt.ylabel("Objective")
        
        plt.subplot(1, 2, 2)
        plt.plot(self.current_parameters, 'ok')
        plt.plot(self.best_parameters, 'r')
        plt.xlabel("Evaluation")
        plt.ylabel("Mode-specific constant")
        
        plt.show()
        
tracker = MyTracker()

Here we start the loop and we let it run until 50 iterations have been obtained. If you want, you can restart the cell to add even more iterations. You should see how the objective value is increasing getting close to zero, which means that the mode share has been pushed towards 30% by calibrating the mode specific constant.

In [ ]:
from octras import Loop

Loop(
   maximum_evaluations = 50 # We run 50x 40 iterations 
).run(
    evaluator = evaluator,
    algorithm = algorithm,
    tracker = tracker
)

The result should look somewhat like this:
<img src="result_corsica.png" />